In [24]:
import os
import sys
sys.path.append('../../')
import pandas as pd
import __init__
from recs.utils.path import *
from recs.utils.common import predict_ranking, readjson2dict, recall_evaluate, id2cat
from lightgbm import LGBMClassifier, LGBMRegressor
from tqdm import tqdm
import numpy as np
import shap

tqdm.pandas()

In [2]:
u2idx=readjson2dict("crm_idx")
i2idx=readjson2dict("fund_info_idx")

In [3]:
pred = pd.read_csv(DATA_PATH / "pred.csv")
pred

,userId,itemId,rating
0,8059,2940,0.202881
1,8059,2936,0.069194
2,8059,2274,0.045018
3,8059,3215,0.026756
4,8059,2937,0.026534
...,...,...,...
519615,64997,1951,0.021333
519616,64997,3063,0.020413
519617,64997,2099,0.017173
519618,64997,3041,0.016063


In [4]:
pred['userId'] = pred['userId'].progress_apply(lambda x: id2cat(u2idx, x))
pred['itemId'] = pred['itemId'].progress_apply(lambda x: id2cat(i2idx, x))
pred

100%|██████████| 519620/519620 [00:56<00:00, 9202.48it/s]


,userId,itemId,rating
0,A2422411050,T38,0.202881
1,A2422411050,T34,0.069194
2,A2422411050,L0C,0.045018
3,A2422411050,Y57,0.026756
4,A2422411050,T35,0.026534
...,...,...,...
519615,P2773542580,H37,0.021333
519616,P2773542580,UY1,0.020413
519617,P2773542580,J1N,0.017173
519618,P2773542580,UN8,0.016063


In [5]:
crm = pd.read_csv(DATA_PATH / "crm_for_clustering_2020.csv")
# crm['身分證字號'] = crm['身分證字號'].apply(lambda x: int(u2idx.get(x, -1)))
crm['userId'] = crm['身分證字號'] 
crm.drop(columns=['身分證字號', 'index'], inplace=True)
crm

Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.


,ETFAUM,外幣定存AUM,海外債AUM,IbrainAUM,投資型保險AUM,年收入,台幣活存AUM,年齡,收入註記,總投資理財AUM,...,職業_38_社工/義工,職業_39_保全業,職業_41_網路拍賣,職業_42_虛擬貨幣業,職業_99_其他,職業_ND,性別_0.0,性別_1.0,性別_2.0,userId
0,0.0,0.000000,0.0,0.0,0.0,13.081541,13.496670,66,1,12.611538,...,0,0,0,0,0,0,0,1,0,A1235199900
1,0.0,0.000000,0.0,0.0,0.0,13.815511,12.279800,66,1,13.787358,...,0,0,0,0,0,0,0,1,0,A1258710430
2,0.0,13.218726,0.0,0.0,0.0,14.919774,11.381688,55,1,14.137175,...,0,0,0,0,0,0,0,1,0,A1520067510
3,0.0,0.000000,0.0,0.0,0.0,0.000000,13.160068,21,0,13.502971,...,0,0,0,0,1,0,0,1,0,A1521553480
4,0.0,0.000000,0.0,0.0,0.0,0.000000,13.239393,21,0,9.210340,...,0,0,0,0,0,0,0,1,0,A1528286510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154948,0.0,0.000000,0.0,0.0,0.0,13.815511,0.000000,48,1,0.000000,...,0,0,0,0,0,0,1,0,0,U1720889160
154949,0.0,0.000000,0.0,0.0,0.0,14.220976,0.000000,32,1,0.000000,...,0,0,0,0,0,0,1,0,0,B1779721610
154950,0.0,0.000000,0.0,0.0,0.0,12.206073,0.000000,49,1,0.000000,...,0,0,0,0,0,0,1,0,0,H2794178210
154951,0.0,0.000000,0.0,0.0,0.0,14.508658,0.000000,43,1,0.000000,...,0,0,0,0,0,0,0,0,1,R2774778510


In [29]:
len(crm.columns)

93

In [6]:
pred = pred.merge(crm, how='left', on=['userId'])
pred

,userId,itemId,rating,ETFAUM,外幣定存AUM,海外債AUM,IbrainAUM,投資型保險AUM,年收入,台幣活存AUM,...,職業_37_旅遊業,職業_38_社工/義工,職業_39_保全業,職業_41_網路拍賣,職業_42_虛擬貨幣業,職業_99_其他,職業_ND,性別_0.0,性別_1.0,性別_2.0
0,A2422411050,T38,0.202881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,A2422411050,T34,0.069194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2,A2422411050,L0C,0.045018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,A2422411050,Y57,0.026756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,A2422411050,T35,0.026534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519635,P2773542580,H37,0.021333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
519636,P2773542580,UY1,0.020413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
519637,P2773542580,J1N,0.017173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
519638,P2773542580,UN8,0.016063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
fund = pd.read_pickle(DATA_PATH / 'fund_info.pkl')
# print(fund.columns.tolist())
fund

,fund_id,fund_type,local_or_foreign,currency_type,chosen,guaranteed,invest_region_沙烏地阿拉伯,invest_region_歐洲不含英國,invest_region_以色列,invest_region_巴西,...,invest_targets_短期債券,invest_targets_股票型,invest_targets_能源,invest_targets_貨幣型,invest_targets_貴重金屬,invest_targets_資訊科技股,invest_targets_農產品,invest_targets_金融股,invest_targets_非必需消費股,invest_targets_高收益債
0,0,3,1,EUR,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3,1,USD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,3,1,USD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,1,EUR,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,4,3,1,USD,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3262,3211,3,1,USD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3263,3212,3,1,EUR,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3264,3213,2,1,USD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3265,3214,2,1,EUR,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
len(fund.columns)

190

In [8]:
fund['itemId'] = fund['fund_id'].progress_apply(lambda x: id2cat(i2idx, x))
fund.drop(columns=['fund_id', 'currency_type'], inplace=True)

100%|██████████| 3267/3267 [00:00<00:00, 9266.01it/s] 


In [9]:
pred = pred.merge(fund, how='left', on=['itemId'])

In [10]:
keys = pred[['userId', 'itemId']]
y = pred['rating']
x = pred.drop(columns=['userId', 'itemId', 'rating', 'CIFAOCODE'])

In [11]:
model = LGBMRegressor(boosting_type="gbdt", verbose=1, random_state=47)
model.fit(x, y)
model.predict([x.values[0]])

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.630852 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6589
[LightGBM] [Info] Number of data points in the train set: 519640, number of used features: 195
[LightGBM] [Info] Start training from score 0.024754


array([0.03577402])

In [12]:

# explainer = shap.KernelExplainer(model.predict, x)
# shap_vals = explainer.shap_values(shap.sample(x, 10000))
explainer = shap.Explainer(model)
shap_vals = explainer(x)

In [13]:
# shap.initjs()
# from pylab import *
# mpl.rcParams['font.sans-serif'] = ['SimHei']
# shap.plots.force(shap_vals[0])

In [13]:
feature_names = shap_vals.feature_names
print(len(feature_names))
# shap_df = pd.DataFrame(shap_vals.values, columns=feature_names)
# vals = np.abs(shap_df.values).mean(0)
# shap_importance = pd.DataFrame(list(zip(feature_names, vals)), columns=['col_name', 'feature_importance_vals'])
# shap_importance

280


In [14]:

value_df_dict = {}
base_val = shap_vals.base_values[0]

for i in tqdm(shap_vals):
    i_sum = i.values.sum()
    for jidx, j in enumerate(feature_names):
        
        data_val = i.data[jidx]
        val = round(i.values[jidx] / (i_sum - base_val), 4)
        save_val = (val, data_val)
        if value_df_dict.get(j, 0) == 0:
            value_df_dict[j] = [save_val]
        else:
            value_df_dict[j].append(save_val)
    

100%|██████████| 519640/519640 [59:03<00:00, 146.64it/s]  


In [47]:
df = pd.DataFrame(value_df_dict)
print(df.columns.tolist())
df

['ETFAUM', '外幣定存AUM', '海外債AUM', 'IbrainAUM', '投資型保險AUM', '年收入', '台幣活存AUM', '年齡', '收入註記', '總投資理財AUM', '外幣庫存註記', 'Line理財標籤註記', '外幣活存AUM', '當年度債券AP', '當年度總AP合計', '投資經驗註記', '台幣定存AUM', '儲蓄型保險AUM', '領股利註記', '存款AUM', '特金AUM', '當年度基金AP', '持有信用卡註記', 'SI/DCI_AUM', '信託戶註記', '保險AUM', '基金AUM', '當年度黃金存摺AP', '當年度人身保險總AP', '海外投資註記', '房貸壽險AUM', '當年度台外幣總存款AP', '當年度(ETF)AP', '業績人員', 'yyyymm', '基金申購註記', '黃金AUM', '保障型保險AUM', '當年度財務部組合存款AP', 'SNAUM', '當年度投資理財總AP', 'AUM', 'has_traded', '職業_0', '職業_0.1', '職業_01_金融業', '職業_02_保險業', '職業_03_投信／投顧／證券', '職業_04_資訊業', '職業_05_建築業', '職業_06_自由業', '職業_07_製造業', '職業_08_貿易買賣業', '職業_09_專業人士', '職業_10_政府機構', '職業_11_各級學校/學術機構', '職業_12_軍警機構', '職業_13_家管/退休人員', '職業_14_軍官/軍人', '職業_15_學生/學齡前兒童', '職業_16_農林漁牧業', '職業_17_礦石及土石採取業', '職業_18_水電燃氣業', '職業_19_批發及零售業', '職業_20_住宿及餐飲業', '職業_21_運輸或倉儲及通訊業', '職業_22_不動產及租賃業', '職業_23_技術服務業', '職業_24_賭博/夜總會相關行業', '職業_25_藝品或寶石交易', '職業_26', '職業_27_無業', '職業_28_軍火商', '職業_29_公職人員', '職業_30_演藝人員', '職業_31_典當業', '職業_32_電子業', '職業_33_大眾傳播', '職業_34_廣告業', '職業_35_營造

,ETFAUM,外幣定存AUM,海外債AUM,IbrainAUM,投資型保險AUM,年收入,台幣活存AUM,年齡,收入註記,總投資理財AUM,...,invest_targets_短期債券,invest_targets_股票型,invest_targets_能源,invest_targets_貨幣型,invest_targets_貴重金屬,invest_targets_資訊科技股,invest_targets_農產品,invest_targets_金融股,invest_targets_非必需消費股,invest_targets_高收益債
0,"(-0.0, 0.0)","(0.0014, 0.0)","(0.0001, 0.0)","(-0.0004, 0.0)","(0.0005, 0.0)","(0.0, 0.0)","(-0.0041, 0.0)","(0.014, 49.0)","(-0.0, 0.0)","(0.001, 0.0)",...,"(-0.0, 0.0)","(0.022, 0.0)","(0.0025, 0.0)","(-0.0, 0.0)","(0.0029, 0.0)","(0.0095, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0024, 0.0)"
1,"(-0.0, 0.0)","(0.0005, 0.0)","(0.0001, 0.0)","(-0.0005, 0.0)","(0.0006, 0.0)","(-0.0012, 0.0)","(-0.0008, 0.0)","(0.0253, 49.0)","(-0.0, 0.0)","(-0.0021, 0.0)",...,"(-0.0, 0.0)","(0.0268, 0.0)","(0.0043, 0.0)","(-0.0, 0.0)","(0.0035, 0.0)","(0.0187, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0049, 0.0)"
2,"(-0.0, 0.0)","(-0.0001, 0.0)","(0.0, 0.0)","(-0.0001, 0.0)","(0.0002, 0.0)","(0.0006, 0.0)","(-0.0003, 0.0)","(0.0069, 49.0)","(-0.0, 0.0)","(-0.001, 0.0)",...,"(-0.0, 0.0)","(0.0079, 0.0)","(0.0012, 0.0)","(-0.0, 0.0)","(0.0032, 0.0)","(0.0021, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0022, 1.0)"
3,"(-0.0, 0.0)","(0.0001, 0.0)","(0.0, 0.0)","(-0.0001, 0.0)","(0.0003, 0.0)","(-0.0003, 0.0)","(-0.0001, 0.0)","(0.0068, 49.0)","(-0.0, 0.0)","(0.0006, 0.0)",...,"(-0.0, 0.0)","(0.0077, 0.0)","(0.0016, 0.0)","(-0.0, 0.0)","(0.003, 0.0)","(0.0139, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0022, 0.0)"
4,"(-0.0, 0.0)","(0.0006, 0.0)","(0.0001, 0.0)","(-0.0001, 0.0)","(0.0003, 0.0)","(-0.0002, 0.0)","(-0.0007, 0.0)","(0.0091, 49.0)","(-0.0, 0.0)","(0.0011, 0.0)",...,"(-0.0, 0.0)","(0.0121, 0.0)","(0.002, 0.0)","(-0.0, 0.0)","(0.0016, 0.0)","(0.0066, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0023, 0.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519635,"(-0.0, 0.0)","(-0.0001, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0002, 0.0)","(-0.0002, 0.0)","(-0.0, 0.0)","(-0.0068, 43.0)","(-0.0, 0.0)","(0.0001, 0.0)",...,"(-0.0, 0.0)","(0.0046, 0.0)","(0.0013, 0.0)","(-0.0, 0.0)","(0.007, 0.0)","(0.0046, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0014, 0.0)"
519636,"(-0.0, 0.0)","(-0.0001, 0.0)","(0.0, 0.0)","(-0.0, 0.0)","(0.0002, 0.0)","(-0.0002, 0.0)","(-0.0001, 0.0)","(-0.0073, 43.0)","(-0.0, 0.0)","(0.0001, 0.0)",...,"(-0.0, 0.0)","(0.0068, 0.0)","(0.0015, 0.0)","(-0.0, 0.0)","(0.0056, 0.0)","(0.0058, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0016, 0.0)"
519637,"(-0.0, 0.0)","(-0.0001, 0.0)","(0.0, 0.0)","(-0.0001, 0.0)","(0.0003, 0.0)","(-0.0002, 0.0)","(-0.0002, 0.0)","(-0.024, 43.0)","(-0.0, 0.0)","(0.0002, 0.0)",...,"(-0.0, 0.0)","(0.0237, 1.0)","(0.0009, 0.0)","(-0.0, 0.0)","(0.0037, 0.0)","(0.0041, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.001, 0.0)"
519638,"(-0.0, 0.0)","(-0.0001, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0002, 0.0)","(-0.0002, 0.0)","(-0.0001, 0.0)","(-0.0098, 43.0)","(-0.0, 0.0)","(0.0002, 0.0)",...,"(-0.0, 0.0)","(0.008, 0.0)","(0.0014, 0.0)","(-0.0, 0.0)","(0.0052, 0.0)","(0.0058, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0016, 0.0)"


In [48]:
df['top5'] = df.apply(lambda x:list(df.columns[np.array(x[::1]).argsort()[::-1][:5]]), axis=1)

In [17]:
df.top5.tolist()

[['currency_美元',
  'fund_type_web_全球型基金',
  'AUM_W',
  'AUM_B',
  'invest_targets_股票型'],
 ['fund_type_web_全球型基金',
  'AUM_W',
  'invest_targets_債券型',
  'invest_targets_股票型',
  '年齡'],
 ['invest_targets_平衡型',
  'currency_美元',
  'invest_region_美國',
  'invest_region_台灣',
  'invest_targets_股票型'],
 ['invest_targets_平衡型',
  'invest_region_美國',
  'invest_targets_資訊科技股',
  'invest_region_全球',
  'invest_region_台灣'],
 ['currency_澳幣',
  'fund_type_web_全球型基金',
  'AUM_W',
  'currency_美元',
  'invest_targets_股票型'],
 ['invest_targets_平衡型',
  'currency_美元',
  'invest_region_美國',
  'invest_region_台灣',
  'invest_targets_股票型'],
 ['invest_targets_債券型',
  'invest_targets_平衡型',
  'invest_region_美國',
  'invest_targets_資訊科技股',
  'invest_targets_生物科技'],
 ['invest_targets_平衡型',
  'invest_targets_債券型',
  'invest_region_美國',
  'invest_targets_資訊科技股',
  'invest_targets_股票型'],
 ['invest_targets_平衡型',
  'invest_targets_債券型',
  'invest_region_美國',
  'invest_targets_資訊科技股',
  'invest_targets_生物科技'],
 ['invest_targets_債券型

In [49]:
df['user'] = keys['userId']
df['item'] = keys['itemId']
df

,ETFAUM,外幣定存AUM,海外債AUM,IbrainAUM,投資型保險AUM,年收入,台幣活存AUM,年齡,收入註記,總投資理財AUM,...,invest_targets_貨幣型,invest_targets_貴重金屬,invest_targets_資訊科技股,invest_targets_農產品,invest_targets_金融股,invest_targets_非必需消費股,invest_targets_高收益債,top5,user,item
0,"(-0.0, 0.0)","(0.0014, 0.0)","(0.0001, 0.0)","(-0.0004, 0.0)","(0.0005, 0.0)","(0.0, 0.0)","(-0.0041, 0.0)","(0.014, 49.0)","(-0.0, 0.0)","(0.001, 0.0)",...,"(-0.0, 0.0)","(0.0029, 0.0)","(0.0095, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0024, 0.0)","[currency_美元, fund_type_web_全球型基金, AUM_W, AUM_...",A2422411050,T38
1,"(-0.0, 0.0)","(0.0005, 0.0)","(0.0001, 0.0)","(-0.0005, 0.0)","(0.0006, 0.0)","(-0.0012, 0.0)","(-0.0008, 0.0)","(0.0253, 49.0)","(-0.0, 0.0)","(-0.0021, 0.0)",...,"(-0.0, 0.0)","(0.0035, 0.0)","(0.0187, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0049, 0.0)","[fund_type_web_全球型基金, AUM_W, invest_targets_債券...",A2422411050,T34
2,"(-0.0, 0.0)","(-0.0001, 0.0)","(0.0, 0.0)","(-0.0001, 0.0)","(0.0002, 0.0)","(0.0006, 0.0)","(-0.0003, 0.0)","(0.0069, 49.0)","(-0.0, 0.0)","(-0.001, 0.0)",...,"(-0.0, 0.0)","(0.0032, 0.0)","(0.0021, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0022, 1.0)","[invest_targets_平衡型, currency_美元, invest_regio...",A2422411050,L0C
3,"(-0.0, 0.0)","(0.0001, 0.0)","(0.0, 0.0)","(-0.0001, 0.0)","(0.0003, 0.0)","(-0.0003, 0.0)","(-0.0001, 0.0)","(0.0068, 49.0)","(-0.0, 0.0)","(0.0006, 0.0)",...,"(-0.0, 0.0)","(0.003, 0.0)","(0.0139, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0022, 0.0)","[invest_targets_平衡型, invest_region_美國, invest_...",A2422411050,Y57
4,"(-0.0, 0.0)","(0.0006, 0.0)","(0.0001, 0.0)","(-0.0001, 0.0)","(0.0003, 0.0)","(-0.0002, 0.0)","(-0.0007, 0.0)","(0.0091, 49.0)","(-0.0, 0.0)","(0.0011, 0.0)",...,"(-0.0, 0.0)","(0.0016, 0.0)","(0.0066, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0023, 0.0)","[currency_澳幣, fund_type_web_全球型基金, AUM_W, curr...",A2422411050,T35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519635,"(-0.0, 0.0)","(-0.0001, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0002, 0.0)","(-0.0002, 0.0)","(-0.0, 0.0)","(-0.0068, 43.0)","(-0.0, 0.0)","(0.0001, 0.0)",...,"(-0.0, 0.0)","(0.007, 0.0)","(0.0046, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0014, 0.0)","[region_LU , invest_region_美國, invest_targets...",P2773542580,H37
519636,"(-0.0, 0.0)","(-0.0001, 0.0)","(0.0, 0.0)","(-0.0, 0.0)","(0.0002, 0.0)","(-0.0002, 0.0)","(-0.0001, 0.0)","(-0.0073, 43.0)","(-0.0, 0.0)","(0.0001, 0.0)",...,"(-0.0, 0.0)","(0.0056, 0.0)","(0.0058, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0016, 0.0)","[invest_region_美國, invest_targets_平衡型, fund_ty...",P2773542580,UY1
519637,"(-0.0, 0.0)","(-0.0001, 0.0)","(0.0, 0.0)","(-0.0001, 0.0)","(0.0003, 0.0)","(-0.0002, 0.0)","(-0.0002, 0.0)","(-0.024, 43.0)","(-0.0, 0.0)","(0.0002, 0.0)",...,"(-0.0, 0.0)","(0.0037, 0.0)","(0.0041, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.001, 0.0)","[invest_targets_平衡型, invest_targets_股票型, fund_...",P2773542580,J1N
519638,"(-0.0, 0.0)","(-0.0001, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0002, 0.0)","(-0.0002, 0.0)","(-0.0001, 0.0)","(-0.0098, 43.0)","(-0.0, 0.0)","(0.0002, 0.0)",...,"(-0.0, 0.0)","(0.0052, 0.0)","(0.0058, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(-0.0, 0.0)","(0.0016, 0.0)","[invest_region_美國, invest_targets_平衡型, fund_ty...",P2773542580,UN8


In [50]:
df.top5 = df.top5.progress_apply(lambda x: ",".join(x))

100%|██████████| 519640/519640 [00:00<00:00, 863325.80it/s]


In [51]:
cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]

In [52]:
df = df[cols]

In [53]:
df.top5

0         currency_美元,fund_type_web_全球型基金,AUM_W,AUM_B,in...
1         fund_type_web_全球型基金,AUM_W,invest_targets_債券型,i...
2         invest_targets_平衡型,currency_美元,invest_region_美...
3         invest_targets_平衡型,invest_region_美國,invest_tar...
4         currency_澳幣,fund_type_web_全球型基金,AUM_W,currency...
                                ...                        
519635    region_LU  ,invest_region_美國,invest_targets_平衡...
519636    invest_region_美國,invest_targets_平衡型,fund_type_...
519637    invest_targets_平衡型,invest_targets_股票型,fund_typ...
519638    invest_region_美國,invest_targets_平衡型,fund_type_...
519639    invest_region_美國,fund_type_web_國內股票開放型科技類,inve...
Name: top5, Length: 519640, dtype: object

In [54]:
df.to_csv('explain.csv')

In [55]:
df.iloc[0]['currency_美元']

(0.0658, 0.0)

In [56]:
df.columns

Index(['user', 'item', 'ETFAUM', '外幣定存AUM', '海外債AUM', 'IbrainAUM', '投資型保險AUM',
       '年收入', '台幣活存AUM', '年齡',
       ...
       'invest_targets_股票型', 'invest_targets_能源', 'invest_targets_貨幣型',
       'invest_targets_貴重金屬', 'invest_targets_資訊科技股', 'invest_targets_農產品',
       'invest_targets_金融股', 'invest_targets_非必需消費股', 'invest_targets_高收益債',
       'top5'],
      dtype='object', length=283)

In [60]:
len(df.user.unique())

51962

In [61]:
len(df.item.unique())

584

In [57]:
df['top5']

0         currency_美元,fund_type_web_全球型基金,AUM_W,AUM_B,in...
1         fund_type_web_全球型基金,AUM_W,invest_targets_債券型,i...
2         invest_targets_平衡型,currency_美元,invest_region_美...
3         invest_targets_平衡型,invest_region_美國,invest_tar...
4         currency_澳幣,fund_type_web_全球型基金,AUM_W,currency...
                                ...                        
519635    region_LU  ,invest_region_美國,invest_targets_平衡...
519636    invest_region_美國,invest_targets_平衡型,fund_type_...
519637    invest_targets_平衡型,invest_targets_股票型,fund_typ...
519638    invest_region_美國,invest_targets_平衡型,fund_type_...
519639    invest_region_美國,fund_type_web_國內股票開放型科技類,inve...
Name: top5, Length: 519640, dtype: object